<b>Bibliotecas</b>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import date, datetime
from ast import literal_eval
import matplotlib.pyplot as plt
import seaborn as sns

<b>API: COVID-19</b>

In [ ]:
url = "https://coronavirus-map.p.rapidapi.com/v1/spots/summary"

headers = {
    'x-rapidapi-host': "coronavirus-map.p.rapidapi.com",
    'x-rapidapi-key': "92274cb262msh0708e2c597ea8bfp1f38e4jsn852f247bcc87"
    }

response = requests.request("GET", url, headers=headers)


In [ ]:
val = literal_eval(response.text)
df_cases = pd.DataFrame.from_dict(val).reset_index().rename(columns={'index': 'day'}).drop(columns=['status', 'type', 'elements'])
df_cases['active_cases'] = 0
df_cases['total_cases'] = 0
df_cases['deaths'] = 0
for item in range(len(df_cases)):
    df_cases['active_cases'][item] =  df_cases['data'][item].get('active_cases')
    df_cases['total_cases'][item] =  df_cases['data'][item].get('total_cases')
    df_cases['deaths'][item] = df_cases['data'][item].get('deaths')

df_cases = df_cases.drop(columns=['data'])
mask = (df_cases['day']== '2020-01-01') | (df_cases['day']== '2020-02-01') | (df_cases['day']== '2020-03-01') | (df_cases['day']== '2020-04-01') | (df_cases['day']== '2020-05-01') | (df_cases['day']== '2020-06-01')
df_cases_day = df_cases[mask].reset_index(drop=True)
df_cases_day

In [ ]:
df_cases

<b>API: Quantidade de acessos - Netflix</b>

In [ ]:
url = "https://similarweb2.p.rapidapi.com/trafficoverview"

querystring = {"website":"https://www.netflix.com"}

headers = {
    'x-rapidapi-host': "similarweb2.p.rapidapi.com",
    'x-rapidapi-key': "92274cb262msh0708e2c597ea8bfp1f38e4jsn852f247bcc87"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

In [ ]:
netflix = literal_eval(response.text)
month_netflix = netflix.get('monthlyVisitsEstimate')
df_netflix = pd.DataFrame.from_dict(month_netflix, orient='index').reset_index().rename(columns={'index':'Data', 0: "Qtd_Acessos"})
mask = df_netflix['Data'] != '2020-01-01'
df_netflix = df_netflix[mask]
df_netflix

In [ ]:
country_netflix = netflix.get("trafficShareByCountry")
df_p_netflix = pd.DataFrame(country_netflix).T
df_p_netflix['country_acessos'] = df_p_netflix.T.sum()
df_p_netflix = df_p_netflix.drop(columns=[0, 1, 2, 3, 4]).reset_index().rename(columns={'index':'country'})
df_p_netflix

<b>API: Quantidade de acessos - Youtube</b>

In [ ]:
url = "https://similarweb2.p.rapidapi.com/trafficoverview"

querystring = {"website":"https://www.youtube.com"}

headers = {
    'x-rapidapi-host': "similarweb2.p.rapidapi.com",
    'x-rapidapi-key': "92274cb262msh0708e2c597ea8bfp1f38e4jsn852f247bcc87"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

In [ ]:
youtube = literal_eval(response.text)
month_youtube = youtube.get('monthlyVisitsEstimate')
df_youtube = pd.DataFrame.from_dict(month_youtube, orient='index').reset_index().rename(columns={'index':'Data', 0: "Qtd_Acessos"})
mask = df_youtube['Data'] != '2020-01-01'
df_youtube = df_youtube[mask]
df_youtube

In [ ]:
country_youtube = youtube.get("trafficShareByCountry")
df_p_youtube = pd.DataFrame(country_youtube).T
df_p_youtube['country_acessos'] = df_p_youtube.T.sum()
df_p_youtube = df_p_youtube.drop(columns=[0, 1, 2, 3, 4]).reset_index().rename(columns={'index':'country'})
df_p_youtube

<b>Base de dados</b>

In [ ]:
acessos = pd.merge(df_netflix, df_youtube, on='Data', how='left', suffixes=('_netflix', '_youtube')).reset_index(drop=True)
acessos

In [ ]:
df_cases_day.to_csv('cases.csv')
acessos.to_csv('acessos.csv')
df_p_youtube.to_csv('paises_youtube.csv')
df_p_netflix.to_csv('paises_netflix.csv')

<b>Gráficos<b>

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(10,9))
ax[0].plot(df_cases_day['day'], df_cases_day['total_cases'], 'black')
ax[0].legend(['Casos de COVID-19'])

ax[1].bar(df_netflix['Data'], df_netflix['Qtd_Acessos'], width=0.25, color = 'b')
ax[1].legend(['Quantidade de acessos na Netflix'])

ax[2].bar(df_youtube['Data'], df_youtube['Qtd_Acessos'], width=0.25, color = 'r')
ax[2].legend(['Quantidade de acessos no Youtube'])
plt.show()